***GENERATED CODE FOR denial16oct2020 PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***OPERATION FUNCTIONS***

In [ ]:


def QueryEditor(spark, joindataobj):
    for source in joindataobj["sourcesForQuery"]:
        sourcename = source['sourcename']
        source['sourcename'] = spark.read.options(header='true', inferschema='true', delimiter=',')\
            .csv('dbfs:' + source['sourcepath'])
        source['sourcename'].createOrReplaceTempView(sourcename)
    joinedDf = spark.sql(joindataobj['querytext'])
    df = removeIfDuplicates(joinedDf)
    print(display(df.limit(2).toPandas()))
    return df


def removeIfDuplicates(joinedDf):
    cols_new = []
    seen = set()
    for c in joinedDf.columns:
        cols_new.append('{}_dup'.format(c) if c in seen else c)
        seen.add(c)
    df = joinedDf.toDF(*cols_new).select(*
                                         [c for c in cols_new if not c.endswith('_dup')])
    return df


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import col, udf, round
from pyspark.ml.feature import Normalizer
from pyspark.sql.types import *
import json
from pyspark.sql.functions import col, round
from pyspark.ml.feature import StandardScaler
from pyspark.sql.functions import col, udf
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import DoubleType, IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def vectorAssemblerTransform(df, param):

    dfReturn = df

    if (type(param) == str):
        outcol = param + "_vector"
        assembler = VectorAssembler(inputCols=[param], outputCol=outcol)
        dfReturn = assembler.transform(dfReturn)
        return dfReturn

    if (type(param) == list):
        vecAssembler = VectorAssembler(inputCols=param, outputCol="features")
        new_df = vecAssembler.transform(df)
        return new_df


def to_array(col):
    def to_array_(v):
        return v.toArray().tolist()
    return udf(to_array_, ArrayType(DoubleType()))(col)


def normalizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']

    dfReturn = dfReturn.fillna({feature: '0.0'})

    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature) \
        .withColumnRenamed("feature_cast", feature)

    outcol = feature + "_normalizer"
    p = transformationData["pNorm"]

    featureVector = feature + "_vector"
    dfReturn = vectorAssemblerTransform(dfReturn, feature)

    normalizer = Normalizer(inputCol=featureVector,
                            outputCol=outcol, p=float(p))
    normalized = normalizer.transform(dfReturn)

    dfReturn = normalized.withColumn("final_col", to_array(normalized[outcol])) \
        .select(normalized.schema.names + [col("final_col")[0]])

    dfReturn = dfReturn.drop(outcol).drop(featureVector)\
        .withColumnRenamed("final_col[0]", outcol)
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[outcol], 2))
    return dfReturn


def standardScalarTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: '0.0'})
    scalarFlags = transformationData["std_scalar"]
    if scalarFlags["mean_flag"]:
        stdflag = False
        meanflag = True
    elif scalarFlags["std_flag"]:
        stdflag = True
        meanflag = False
    outcol = feature + "_standardscalar"
    featureVector = feature + "_vector"
    dfReturn = vectorAssemblerTransform(dfReturn, feature)

    standardscale = StandardScaler(inputCol=featureVector, outputCol=outcol, withStd=stdflag,
                                   withMean=meanflag)
    scaledata = standardscale.fit(dfReturn).transform(dfReturn)
    dfReturn = scaledata.withColumn("final_col", to_array(scaledata[outcol]))\
        .select(scaledata.schema.names + [col("final_col")[0]])

    dfReturn = dfReturn.drop(outcol).drop(featureVector)\
        .withColumnRenamed("final_col[0]", outcol)
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[outcol], 2))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PatientID', 'transformation_label': 'String Indexer'}], 'feature': 'PatientID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '9898', 'mean': '', 'stddev': '', 'min': '0001566B-77D5-45F2-A019-E764463B0F03', 'max': 'FFF654C0-B358-4D4B-99CE-53F3B1BC3442', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PatientID'}, {'feature_label': 'PatientID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PatientID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '9898', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'O', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'City', 'transformation_label': 'String Indexer'}], 'feature': 'City', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '9898', 'mean': '', 'stddev': '', 'min': 'Carrollton', 'max': 'Yorktown', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'City'}, {'feature_label': 'City', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('City')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '9898', 'mean': '', 'stddev': '', 'min': 'After Hours Care Newport News', 'max': 'Yorktown Family Medicine', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityCategory'}, {'feature_label': 'FacilityCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderID', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '9898', 'mean': '', 'stddev': '', 'min': '000C959E-D188-4091-96D1-6AA6D81562F6', 'max': 'FF2C0B65-05D1-4527-8C11-8E1BAC8C12EA', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProviderID'}, {'feature_label': 'ProviderID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PracticeCategory', 'transformation_label': 'String Indexer'}], 'feature': 'PracticeCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '9898', 'mean': '', 'stddev': '', 'min': 'Charles Lee Ginsburgh', 'max': 'TPMG', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PracticeCategory'}, {'feature_label': 'PracticeCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PracticeCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '9898', 'mean': '', 'stddev': '', 'min': '<Unspecified>', 'max': 'X-Ray', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentCategory'}, {'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepartmentCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '9898', 'mean': '89173.38', 'stddev': '131666.82', 'min': '00731', 'max': 'S0020', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTCode'}, {'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CPTCode')
        transformationDF = normalizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PatientBal', 'pNorm': '2.0', 'transformation_label': 'Normalizer'}], 'feature': 'PatientBal', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '9898', 'mean': '2.56', 'stddev': '38.82', 'min': '-466.73', 'max': '2711.88', 'missing': '0'}, 'transformation': [{'transformation': 'Normalizer', 'selectedAsDefault': 1}], 'updatedLabel': 'PatientBal'}, {'feature_label': 'PatientBal', 'pNorm': '2.0', 'transformation_label': 'Normalizer'})
        transformationDF = transformationDF.drop('PatientBal')
        transformationDF = normalizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryInsuranceBal', 'pNorm': '2.0', 'transformation_label': 'Normalizer'}], 'feature': 'PrimaryInsuranceBal', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '9898', 'mean': '1.81', 'stddev': '25.57', 'min': '-251.0', 'max': '1413.75', 'missing': '0'}, 'transformation': [{'transformation': 'Normalizer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryInsuranceBal'}, {'feature_label': 'PrimaryInsuranceBal', 'pNorm': '2.0', 'transformation_label': 'Normalizer'})
        transformationDF = transformationDF.drop('PrimaryInsuranceBal')
        transformationDF = normalizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondaryInsuranceBal', 'pNorm': '2.0', 'transformation_label': 'Normalizer'}], 'feature': 'SecondaryInsuranceBal', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '9898', 'mean': '0.12', 'stddev': '6.66', 'min': '-391.0', 'max': '251.0', 'missing': '0'}, 'transformation': [{'transformation': 'Normalizer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondaryInsuranceBal'}, {'feature_label': 'SecondaryInsuranceBal', 'pNorm': '2.0', 'transformation_label': 'Normalizer'})
        transformationDF = transformationDF.drop('SecondaryInsuranceBal')
        transformationDF = normalizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TertiaryInsuranceBal', 'pNorm': '2.0', 'transformation_label': 'Normalizer'}], 'feature': 'TertiaryInsuranceBal', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '9898', 'mean': '0.0', 'stddev': '1.47', 'min': '-73.41', 'max': '104.09', 'missing': '0'}, 'transformation': [{'transformation': 'Normalizer', 'selectedAsDefault': 1}], 'updatedLabel': 'TertiaryInsuranceBal'}, {'feature_label': 'TertiaryInsuranceBal', 'pNorm': '2.0', 'transformation_label': 'Normalizer'})
        transformationDF = transformationDF.drop('TertiaryInsuranceBal')
        transformationDF = standardScalarTransform(transformationDF, {'transformationsData': [{'feature_label': 'Diagnosis1', 'std_scalar': {'mean_flag': 'True', 'std_flag': 'False'}, 'transformation_label': 'Standard Scalar'}], 'feature': 'Diagnosis1', 'transformation': [{'transformation': 'Standard Scalar', 'selectedAsDefault': 1}], 'type': 'numeric', 'replaceby': 'mean', 'selected': 'True', 'stats': {
            'count': '9898', 'mean': '34264.84', 'stddev': '19744.9', 'min': '859', 'max': '94904', 'missing': '0'}, 'updatedLabel': 'Diagnosis1'}, {'feature_label': 'Diagnosis1', 'std_scalar': {'mean_flag': 'True', 'std_flag': 'False'}, 'transformation_label': 'Standard Scalar'})
        transformationDF = transformationDF.drop('Diagnosis1')
        transformationDF = standardScalarTransform(transformationDF, {'transformationsData': [{'feature_label': 'Diagnosis2', 'std_scalar': {'mean_flag': 'True', 'std_flag': 'False'}, 'transformation_label': 'Standard Scalar'}], 'feature': 'Diagnosis2', 'transformation': [{'transformation': 'Standard Scalar', 'selectedAsDefault': 1}], 'type': 'numeric', 'replaceby': 'mean', 'selected': 'True', 'stats': {
            'count': '9898', 'mean': '36690.6', 'stddev': '21641.42', 'min': '266', 'max': '94919', 'missing': '0'}, 'updatedLabel': 'Diagnosis2'}, {'feature_label': 'Diagnosis2', 'std_scalar': {'mean_flag': 'True', 'std_flag': 'False'}, 'transformation_label': 'Standard Scalar'})
        transformationDF = transformationDF.drop('Diagnosis2')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run denial16oct2020Hooks.ipynb
try:
	#sourcePreExecutionHook()
	denial = RDBMSConnector.fetch(spark, "{'url': '/FileStore/platform/sampleData/5f897f948839575ad147aa4f/part-00000-tid-3688035541454563243-bcfc9663-95e2-4a2c-9fee-c6e4f8a284db-14402-1-c000.csv', 'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'NextGen_Tidewater_Physicians', 'optionalDB': 'NextGen_Tidewater_Physicians', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT DISTINCT TOP(100000) \r\ncd.PatientID, pat.Gender, FLOOR(DATEDIFF(DAY, pat.DateOfBirth, GETDATE()) / 365.25) AS Age, fac.City, \r\nMonth(cd.DateOfEntry) AS MonthOfEntry, Year(cd.DateOfEntry) as YearOfEntry, fac.FacilityCategory, cd.Unit,\r\ncd.ProviderID, Pr.PracticeCategory, dep.DepartmentCategory, cp.CPTCode, cd.Amount,\r\ncd.PatientBal, cd.PrimaryInsuranceBal, cd.SecondaryInsuranceBal, cd.TertiaryInsuranceBal, cd.DiagnosisCodeID1 AS Diagnosis1,\r\ncd.DiagnosisCodeID2 AS Diagnosis2, status=(SELECT CASE trc.ReasonType WHEN 'Denial' THEN 1 ELSE 0 END)\r\nFROM chargedetail cd inner join TransactionReason as tr on tr.ChargeDetailID = cd.ChargeDetailID\r\ninner join TransactionReasonCode as trc on tr.TransactionReasonCodeID = trc.TransactionReasonCodeID \r\ninner join Practice as Pr on Pr.PracticeID = cd.PracticeID  \r\ninner join CptCode as cp on cp.CPTCodeID = cd.CPTCodeID  \r\ninner join Department as dep on dep.DepartmentID = cd.DepartmentID  \r\ninner join Facility as fac on fac.FacilityID = cd.FacilityID  \r\ninner join Patient as pat on pat.PatientID = cd.PatientID  \r\nWHERE trc.ReasonType IN ('Denial') \r\nand cd.Amount > .01 and\r\nexists ( select distinct ChargeDetailID as cd from TransactionReasonCode as tr inner join TransactionReason as r on tr.TransactionReasonCodeID = r.TransactionReasonCodeID and cd.ChargeDetailID = r.ChargeDetailID)", 'dbtype': 'mssql', 'is_header': 'Use Header Line'}")



	nondenial = RDBMSConnector.fetch(spark, "{'url': '/FileStore/platform/sampleData/5f897f968839575ad147aa50/part-00000-tid-5497748112589743471-f20cf046-6164-4614-a6f5-a2489976e6c3-14406-1-c000.csv', 'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'NextGen_Tidewater_Physicians', 'optionalDB': 'NextGen_Tidewater_Physicians', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT DISTINCT TOP(100000) \r\ncd.PatientID, pat.Gender, FLOOR(DATEDIFF(DAY, pat.DateOfBirth, GETDATE()) / 365.25) AS Age, fac.City, \r\nMonth(cd.DateOfEntry) AS MonthOfEntry, Year(cd.DateOfEntry) as YearOfEntry, fac.FacilityCategory, cd.Unit,\r\ncd.ProviderID, Pr.PracticeCategory, dep.DepartmentCategory, cp.CPTCode, cd.Amount,\r\ncd.PatientBal, cd.PrimaryInsuranceBal, cd.SecondaryInsuranceBal, cd.TertiaryInsuranceBal, cd.DiagnosisCodeID1 AS Diagnosis1,\r\ncd.DiagnosisCodeID2 AS Diagnosis2, status=(SELECT CASE trc.ReasonType WHEN 'Denial' THEN 1 ELSE 0 END)\r\nFROM chargedetail cd inner join TransactionReason as tr on tr.ChargeDetailID = cd.ChargeDetailID\r\ninner join TransactionReasonCode as trc on tr.TransactionReasonCodeID = trc.TransactionReasonCodeID \r\ninner join Practice as Pr on Pr.PracticeID = cd.PracticeID  \r\ninner join CptCode as cp on cp.CPTCodeID = cd.CPTCodeID  \r\ninner join Department as dep on dep.DepartmentID = cd.DepartmentID  \r\ninner join Facility as fac on fac.FacilityID = cd.FacilityID  \r\ninner join Patient as pat on pat.PatientID = cd.PatientID  \r\nWHERE trc.ReasonType IN ('Contractual', 'Informational', 'Co-Ins/Deduct', 'SPAY-Patient Responsibility', 'Other') \r\nand cd.Amount > .01 and\r\nexists ( select distinct ChargeDetailID as cd from TransactionReasonCode as tr inner join TransactionReason as r on tr.TransactionReasonCodeID = r.TransactionReasonCodeID and cd.ChargeDetailID = r.ChargeDetailID)", 'dbtype': 'mssql', 'is_header': 'Use Header Line'}")
	#sourcePostExecutionHook(nondenial)

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run denial16oct2020Hooks.ipynb
try:
	#operationPreExecutionHook()

	joindata = QueryEditor(spark, {"querytext": "SELECT  * FROM Denial UNION ALL Select *  FROM NonDenial ", "sourcesForQuery": [{"sourcename": "NonDenial", "sourcepath": "/FileStore/platform/sampleData/5f897f968839575ad147aa50/part-00000-tid-5497748112589743471-f20cf046-6164-4614-a6f5-a2489976e6c3-14406-1-c000.csv", "sourceid": "5f897f968839575ad147aa50"}, {"sourcename": "Denial", "sourcepath": "/FileStore/platform/sampleData/5f897f948839575ad147aa4f/part-00000-tid-3688035541454563243-bcfc9663-95e2-4a2c-9fee-c6e4f8a284db-14402-1-c000.csv", "sourceid": "5f897f948839575ad147aa4f"}], "fieldsForQuery": []})
	#operationPostExecutionHook(joindata)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run denial16oct2020Hooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(joindata,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "PatientID", "transformation_label": "String Indexer"}], "feature": "PatientID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "9898", "mean": "", "stddev": "", "min": "0001566B-77D5-45F2-A019-E764463B0F03", "max": "FFF654C0-B358-4D4B-99CE-53F3B1BC3442", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PatientID"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "9898", "mean": "", "stddev": "", "min": "F", "max": "O", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "9898", "mean": "63.91", "stddev": "18.19", "min": "0", "max": "104", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "City", "transformation_label": "String Indexer"}], "feature": "City", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "9898", "mean": "", "stddev": "", "min": "Carrollton", "max": "Yorktown", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "City"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MonthOfEntry", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "9898", "mean": "6.15", "stddev": "3.4", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "MonthOfEntry"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "YearOfEntry", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "9898", "mean": "2017.93", "stddev": "1.39", "min": "2013", "max": "2020", "missing": "0"}, "updatedLabel": "YearOfEntry"}, {"transformationsData": [{"feature_label": "FacilityCategory", "transformation_label": "String Indexer"}], "feature": "FacilityCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "9898", "mean": "", "stddev": "", "min": "After Hours Care Newport News", "max": "Yorktown Family Medicine", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityCategory"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Unit", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "9898", "mean": "1.09", "stddev": "1.82", "min": "1.0", "max": "99.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Unit"}, {"transformationsData": [{"feature_label": "ProviderID", "transformation_label": "String Indexer"}], "feature": "ProviderID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "9898", "mean": "", "stddev": "", "min": "000C959E-D188-4091-96D1-6AA6D81562F6", "max": "FF2C0B65-05D1-4527-8C11-8E1BAC8C12EA", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProviderID"}, {"transformationsData": [{"feature_label": "PracticeCategory", "transformation_label": "String Indexer"}], "feature": "PracticeCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "9898", "mean": "", "stddev": "", "min": "Charles Lee Ginsburgh", "max": "TPMG", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PracticeCategory"}, {"transformationsData": [{"feature_label": "DepartmentCategory", "transformation_label": "String Indexer"}], "feature": "DepartmentCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "9898", "mean": "", "stddev": "", "min": "<Unspecified>", "max": "X-Ray", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartmentCategory"}, {"transformationsData": [{"feature_label": "CPTCode", "transformation_label": "String Indexer"}], "feature": "CPTCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "9898", "mean": "89173.38", "stddev": "131666.82", "min": "00731", "max": "S0020", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CPTCode"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "9898", "mean": "180.78", "stddev": "283.09", "min": "4.0", "max": "6300.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount"}, {"transformationsData": [{"feature_label": "PatientBal", "pNorm": "2.0", "transformation_label": "Normalizer"}], "feature": "PatientBal", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "9898", "mean": "2.56", "stddev": "38.82", "min": "-466.73", "max": "2711.88", "missing": "0"}, "transformation": [{"transformation": "Normalizer", "selectedAsDefault": 1}], "updatedLabel": "PatientBal"}, {"transformationsData": [{"feature_label": "PrimaryInsuranceBal", "pNorm": "2.0", "transformation_label": "Normalizer"}], "feature": "PrimaryInsuranceBal", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "9898", "mean": "1.81", "stddev": "25.57", "min": "-251.0", "max": "1413.75", "missing": "0"}, "transformation": [{"transformation": "Normalizer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryInsuranceBal"}, {"transformationsData": [{"feature_label": "SecondaryInsuranceBal", "pNorm": "2.0", "transformation_label": "Normalizer"}], "feature": "SecondaryInsuranceBal", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "9898", "mean": "0.12", "stddev": "6.66", "min": "-391.0", "max": "251.0", "missing": "0"}, "transformation": [{"transformation": "Normalizer", "selectedAsDefault": 1}], "updatedLabel": "SecondaryInsuranceBal"}, {"transformationsData": [{"feature_label": "TertiaryInsuranceBal", "pNorm": "2.0", "transformation_label": "Normalizer"}], "feature": "TertiaryInsuranceBal", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "9898", "mean": "0.0", "stddev": "1.47", "min": "-73.41", "max": "104.09", "missing": "0"}, "transformation": [{"transformation": "Normalizer", "selectedAsDefault": 1}], "updatedLabel": "TertiaryInsuranceBal"}, {"transformationsData": [{"feature_label": "Diagnosis1", "std_scalar": {"mean_flag": "True", "std_flag": "False"}, "transformation_label": "Standard Scalar"}], "feature": "Diagnosis1", "transformation": [{"transformation": "Standard Scalar", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "9898", "mean": "34264.84", "stddev": "19744.9", "min": "859", "max": "94904", "missing": "0"}, "updatedLabel": "Diagnosis1"}, {"transformationsData": [{"feature_label": "Diagnosis2", "std_scalar": {"mean_flag": "True", "std_flag": "False"}, "transformation_label": "Standard Scalar"}], "feature": "Diagnosis2", "transformation": [{"transformation": "Standard Scalar", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "9898", "mean": "36690.6", "stddev": "21641.42", "min": "266", "max": "94919", "missing": "0"}, "updatedLabel": "Diagnosis2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "status", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "9898", "mean": "0.48", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "status"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run denial16oct2020Hooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["PatientID_stringindexer", "Gender_stringindexer", "Age", "City_stringindexer", "MonthOfEntry", "YearOfEntry", "FacilityCategory_stringindexer", "Unit", "ProviderID_stringindexer", "PracticeCategory_stringindexer", "DepartmentCategory_stringindexer", "CPTCode_stringindexer", "Amount", "PatientBal_normalizer", "PrimaryInsuranceBal_normalizer", "SecondaryInsuranceBal_normalizer", "TertiaryInsuranceBal_normalizer", "Diagnosis1_standardscalar", "Diagnosis2_standardscalar"], "status")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

